In [1]:
pwd

'C:\\Users\\Dell'

In [2]:
cd downloads

C:\Users\Dell\downloads


In [3]:
cd character-extraction-master

C:\Users\Dell\downloads\character-extraction-master


In [4]:
cd character-extraction-master

C:\Users\Dell\downloads\character-extraction-master\character-extraction-master


In [5]:
import json
import nltk
import re

from collections import defaultdict
from nltk.corpus import stopwords
from pattern.en import parse, Sentence, mood
from pattern.db import csv
from pattern.vector import Document, NB

In [55]:
def readText():
    """
    Reads the text from a text file.
    """
    with open("hp_ps.txt", "rb") as f:
        text = f.read().decode('utf-8-sig')
    return text

In [7]:
def chunkSentences(text):
    """
    Parses text into parts of speech tagged with parts of speech labels.

    Used for reference: https://gist.github.com/onyxfish/322906
    """
    sentences = nltk.sent_tokenize(text)
    tokenizedSentences = [nltk.word_tokenize(sentence)
                          for sentence in sentences]
    taggedSentences = [nltk.pos_tag(sentence)
                       for sentence in tokenizedSentences]
    if nltk.__version__[0:2] == "2.":
        chunkedSentences = nltk.batch_ne_chunk(taggedSentences, binary=True)
    else:
        chunkedSentences = nltk.ne_chunk_sents(taggedSentences, binary=True)
    return chunkedSentences

In [8]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
def extractEntityNames(tree, _entityNames=None):
    """
    Creates a local list to hold nodes of tree passed through, extracting named
    entities from the chunked sentences.

    Used for reference: https://gist.github.com/onyxfish/322906
    """
    if _entityNames is None:
        _entityNames = []
    try:
        if nltk.__version__[0:2] == "2.":
            label = tree.node
        else:
            label = tree.label()
    except AttributeError:
        pass
    else:
        if label == 'NE':
            _entityNames.append(' '.join([child[0] for child in tree]))
        else:
            for child in tree:
                extractEntityNames(child, _entityNames=_entityNames)
    return _entityNames

In [10]:
def buildDict(chunkedSentences, _entityNames=None):
    """
    Uses the global entity list, creating a new dictionary with the properties
    extended by the local list, without overwriting.

    Used for reference: https://gist.github.com/onyxfish/322906
    """
    if _entityNames is None:
        _entityNames = []

    for tree in chunkedSentences:
        extractEntityNames(tree, _entityNames=_entityNames)

    return _entityNames

In [11]:
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [12]:
def removeStopwords(entityNames, customStopWords=None):
    """
    Brings in stopwords and custom stopwords to filter mismatches out.
    """
    # Memoize custom stop words
    if customStopWords is None:
        with open("customStopWords.txt", "rb") as f:
            customStopwords = f.read().split(', ')

    for name in entityNames:
        if name in stopwords.words('english') or name in customStopwords:
            entityNames.remove(name)

In [13]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [14]:
def getMajorCharacters(entityNames):
    """
    Adds names to the major character list if they appear frequently.
    """
    return {name for name in entityNames if entityNames.count(name) > 10}

In [15]:
def splitIntoSentences(text):
    """
    Split sentences on .?! "" and not on abbreviations of titles.
    Used for reference: http://stackoverflow.com/a/8466725
    """
    sentenceEnders = re.compile(r"""
    # Split sentences on whitespace between them.
    (?:               # Group for two positive lookbehinds.
      (?<=[.!?])      # Either an end of sentence punct,
    | (?<=[.!?]['"])  # or end of sentence punct and quote.
    )                 # End group of two positive lookbehinds.
    (?<!  Mr\.   )    # Don't end sentence on "Mr."
    (?<!  Mrs\.  )    # Don't end sentence on "Mrs."
    (?<!  Ms\.   )    # Don't end sentence on "Ms."
    (?<!  Jr\.   )    # Don't end sentence on "Jr."
    (?<!  Dr\.   )    # Don't end sentence on "Dr."
    (?<!  Prof\. )    # Don't end sentence on "Prof."
    (?<!  Sr\.   )    # Don't end sentence on "Sr."
    \s+               # Split on whitespace between sentences.
    """, re.IGNORECASE | re.VERBOSE)
    return sentenceEnders.split(text)

In [16]:
def compareLists(sentenceList, majorCharacters):
    """
    Compares the list of sentences with the character names and returns
    sentences that include names.
    """
    characterSentences = defaultdict(list)
    for sentence in sentenceList:
        for name in majorCharacters:
            if re.search(r"\b(?=\w)%s\b(?!\w)" % re.escape(name),
                         sentence,
                         re.IGNORECASE):
                characterSentences[name].append(sentence)
    return characterSentences


In [17]:
def extractMood(characterSentences):
    """
    Analyzes the sentence using grammatical mood module from pattern.
    """
    characterMoods = defaultdict(list)
    for key, value in characterSentences.items():
        for x in value:
            characterMoods[key].append(mood(Sentence(parse(str(x),
                                                           lemmata=True))))
    return characterMoods

In [28]:
def extractSentiment(characterSentences):
    """
    Trains a Naive Bayes classifier object with the reviews.csv file, analyzes
    the sentence, and returns the tone.
    """
    nb = NB()
    characterTones = defaultdict(list)
    for review, rating in csv("reviews.csv"):
        nb.train(Document(review, type=int(rating), stopwords=True))
    for key, value in characterSentences.items():
        for x in value:
            characterTones[key].append(nb.classify(str(x)))
    return characterTones

In [29]:
def writeAnalysis(sentenceAnalysis):
    """
    Writes the sentence analysis to a text file in the same directory.
    """
    with open("sentenceAnalysis.txt", "wb") as f:
        for item in sentenceAnalysis.items():
            f.write("%s:%s\n" % item)

In [30]:
def writeToJSON(sentenceAnalysis):
    """
    Writes the sentence analysis to a JSON file in the same directory.
    """
    with open("sentenceAnalysis.json", "wb") as f:
        json.dump(sentenceAnalysis, f)

In [31]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [32]:
!pip install many_stop_words

  Stored in directory: C:\Users\Dell\AppData\Local\pip\Cache\wheels\cb\70\60\6cd0b179da831dc80b5b07bd58dbf4cd99fa82b289eb41ada8
Successfully built many-stop-words


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [35]:
from many_stop_words import get_stop_words
stop_list=list(get_stop_words('en'))        #About 900 stop words
nltk_words = list(stopwords.words('english'))   #About 150 stop words
stop_list.extend(nltk_words)

In [38]:
additional_stopwords = """AND THE WHICH Mr."""

# Split the the additional stopwords string on each word and then add
# those words to the NLTK stopwords list
stop_list += additional_stopwords.split()

In [56]:
text = readText()

In [63]:
majorCharacters

{'Aunt Petunia',
 'Bane',
 'Charlie',
 'Crabbe',
 'Dudley',
 'Dumbledore',
 'Dursleys',
 'Everyone',
 'Fang',
 'Filch',
 'Firenze',
 'Flamel',
 'Fluffy',
 'Fred',
 'George',
 'Goyle',
 'Great Hall',
 'Gringotts',
 'Griphook',
 'Gryffindor',
 'Hagrid',
 'Harry',
 'Harry Potter',
 'Hedwig',
 'Hermione',
 'Hermione Granger',
 'Hogwarts',
 'House',
 'Hufflepuff',
 'Invisibility Cloak',
 'London',
 'Madam Hooch',
 'Madam Pomfrey',
 'Magic',
 'Malfoy',
 'McGonagall',
 'Mr. Dursley',
 'Mr. Ollivander',
 'Mr. Potter',
 'Muggle',
 'Muggles',
 'Neville',
 'Norbert',
 'Page',
 'Peeves',
 'Percy',
 'Philosophers Stone',
 'Potter',
 'Privet Drive',
 'Professor',
 'Professor McGonagall',
 'Quaffle',
 'Quidditch',
 'Quirrell',
 'Ron',
 'Ronan',
 'Seamus',
 'Slytherin',
 'Snape',
 'Snitch',
 'Sorcerer',
 'Uncle Vernon',
 'Voldemort',
 'Weasley',
 'Wood'}

In [58]:
chunkedSentences = chunkSentences(text)


In [59]:
entityNames = buildDict(chunkedSentences)

In [60]:
len(entityNames)

5736

In [121]:
 majorCharacters = getMajorCharacters(entityNames)
len(majorCharacters)
majCharacters=list(majorCharacters)

In [62]:
entNames = list(set(entityNames))
len(entNames)

638

In [52]:
with open('customStopWords.txt') as stopfile:
    stopwords = stopfile.read()
    list1 = stopwords.split(',')

In [64]:
for name in majorCharacters:
        if name in list1:
            entityNames.remove(name)

In [65]:
for name in majorCharacters:
        if name in stop_list:
            entityNames.remove(name)

In [69]:
type(majorCharacters)#set
s=list(majorCharacters)

In [70]:
sentenceList = splitIntoSentences(text)

In [180]:
characterSentences = compareLists(sentenceList, majorCharacters)

In [78]:
characterMoods = defaultdict(list)
for key, value in characterSentences.items():
        for x in value:
            continue
            #characterMoods[key].append(mood(Sentence(parse(str(x)))))

In [79]:
sentenceAnalysis = defaultdict(list,
                                   [(k, [characterSentences[k],
                                         characterTones[k]])
                                    for k in characterSentences])

In [82]:
type(characterSentences)

collections.defaultdict

In [104]:
with open("sentenceAnalysis_hp.txt", "w") as f:
       for item in sentenceAnalysis.items():
            #f.write("%s:%s\n" % item)
            f.write(item[0])
            f.write("   :   ")
            for row in item[1]:
                f.write("\n")
        


In [99]:
with open("sentenceAnalysis.json", "w") as f:
        json.dump(sentenceAnalysis, f)

In [106]:
from bookworm import *
#!pip install spacy 2.2.4

In [107]:
#book = load_book('hp_ps.txt', lower=True)
!pip install python-louvain

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [108]:
sequences = get_sentence_sequences(text)

In [113]:
df = find_connections(sequences, majorCharacters)

In [115]:
cooccurence = calculate_cooccurence(df)

In [116]:
cooccurence

,Harry Potter,Mr. Ollivander,Mr. Dursley,Norbert,Hermione Granger,Quirrell,London,Firenze,Hermione,Ron,...,Harry,Page,George,Professor McGonagall,Griphook,Weasley,Everyone,Goyle,Ronan,Fred
Harry Potter,0,29305918,22753787,15067883,41261541,14792011,11646815,14928511,18561486,5125237,...,8464233,7973974,14330885,42315493,12849806,15459575,16792046,9384923,10090737,8091106
Mr. Ollivander,29305918,0,21026883,13804501,38134433,13734640,10770987,13838749,17160327,4728889,...,7777715,7382176,13207097,39110187,11814128,14347612,15502031,8681381,9351581,7484010
Mr. Dursley,22753787,21026883,0,10738369,29527829,10653442,8301503,10711985,13293103,3649100,...,6041139,5705740,10264097,30309797,9159585,11134314,12034658,6744198,7189998,5804574
Norbert,15067883,13804501,10738369,0,19493792,7003245,5501178,7074967,8797006,2421386,...,3982792,3753645,6808701,19947670,6078574,7297280,7964454,4446149,4739756,3840017
Hermione Granger,41261541,38134433,29527829,19493792,0,19249629,15184326,19535641,24236988,6677487,...,10923492,10392754,18677707,54930035,16648253,20158755,21902281,12206759,13159272,10550250
Quirrell,14792011,13734640,10653442,7003245,19249629,0,5384501,7011814,8682559,2363929,...,3937544,3713816,6699421,19716248,5963031,7261932,7845680,4404543,4660437,3797457
London,11646815,10770987,8301503,5501178,15184326,5384501,0,5480598,6857435,1972839,...,3046953,2905938,5260339,15650544,4790706,5628825,6197232,3471879,3831244,2951219
Firenze,14928511,13838749,10711985,7074967,19535641,7011814,5480598,0,8835973,2406734,...,3937089,3777820,6807926,19848238,6010002,7352907,7985343,4435315,4747165,3854785
Hermione,18561486,17160327,13293103,8797006,24236988,8682559,6857435,8835973,0,3016073,...,4888920,4673458,8447750,24715073,7513966,9090655,9904556,5520077,5916960,4765465
Ron,5125237,4728889,3649100,2421386,6677487,2363929,1972839,2406734,3016073,0,...,1338222,1276399,2314081,6893061,2112586,2469063,2726893,1528735,1691880,1289417


In [117]:
cooccurence = cooccurence.to_sparse()

In [118]:
def print_five_closest(character):
    print('-'*len(str(character))
          + '\n' + str(character) + '\n'
          + '-'*len(str(character)))
    
    top_five = (cooccurence[str(character)]
                .sort_values(ascending=False)
                .index.values
                [:5])
    
    for name in top_five:
        print(name)

In [123]:
from random import randint
for i in range(5):
    print_five_closest(majCharacters[randint(0, len(majCharacters))])
    print()

-----
House
-----
Philosophers Stone
Professor McGonagall
Hermione Granger
Invisibility Cloak
Harry Potter

--------
Quirrell
--------
Philosophers Stone
Professor McGonagall
Hermione Granger
Invisibility Cloak
Harry Potter

---------
Quidditch
---------
Philosophers Stone
Professor McGonagall
Hermione Granger
Invisibility Cloak
Harry Potter

--------
Quirrell
--------
Philosophers Stone
Professor McGonagall
Hermione Granger
Invisibility Cloak
Harry Potter

--------
Sorcerer
--------
Philosophers Stone
Professor McGonagall
Hermione Granger
Invisibility Cloak
Harry Potter



In [125]:
print_five_closest('Ron')
print_five_closest('Hermione Granger')
print_five_closest('Harry Potter')

---
Ron
---
Philosophers Stone
Professor McGonagall
Hermione Granger
Invisibility Cloak
Harry Potter
----------------
Hermione Granger
----------------
Philosophers Stone
Professor McGonagall
Invisibility Cloak
Harry Potter
Uncle Vernon
------------
Harry Potter
------------
Philosophers Stone
Professor McGonagall
Hermione Granger
Invisibility Cloak
Uncle Vernon


In [126]:
!python -m spacy download en

[!] Skipping model package dependencies and setting `--no-deps`. You don't seem
to have the spaCy package itself installed (maybe because you've built from
source?), so installing the model dependencies would cause spaCy to be
downloaded, which probably isn't what you want. If the model package has other
dependencies, you'll have to install them manually.
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[x] Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\Dell\Anaconda3\lib\site-packages\en_core_web_sm -->
C:\Users\Dell\Anaconda3\lib\site-packages\spacy\data\en
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you ca

You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You do not have sufficient privilege to perform this operation.


In [127]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [131]:
def getMajorCharacters(entityNames):
    """
    Adds names to the major character list if they appear frequently.
    """
    return {name for name in entityNames if entityNames.count(name) > 15}

In [129]:
for name in majorCharacters:
        print(entityNames.count(name))
        print(name)

368
Harry Potter
16
Mr. Ollivander
27
Mr. Dursley
22
Norbert
13
Hermione Granger
78
Quirrell
12
London
15
Firenze
192
Hermione
338
Ron
106
Malfoy
48
Aunt Petunia
28
Potter
14
Seamus
327
Hagrid
14
Flamel
18
Crabbe
33
Slytherin
15
McGonagall
19
Fluffy
14
Privet Drive
20
House
15
Snitch
115
Snape
12
Hedwig
25
Voldemort
15
Great Hall
16
Fang
12
Bane
103
Dumbledore
96
Uncle Vernon
35
Hogwarts
19
Gringotts
52
Gryffindor
14
Peeves
11
Madam Hooch
11
Hufflepuff
43
Filch
347
Philosophers Stone
104
Neville
12
Magic
26
Percy
23
Professor
17
Quaffle
43
Quidditch
45
Wood
12
Invisibility Cloak
12
Mr. Potter
44
Dursleys
14
Muggles
14
Muggle
127
Dudley
13
Madam Pomfrey
12
Charlie
15
Sorcerer
1155
Harry
192
Page
18
George
25
Professor McGonagall
11
Griphook
19
Weasley
12
Everyone
22
Goyle
15
Ronan
22
Fred


In [132]:
majorCharacters1=getMajorCharacters(entityNames)

In [134]:
len(majorCharacters1)

40

In [148]:
lines_list = open('pa.txt').read().splitlines()

In [152]:
type(lines_list[0])

str

In [183]:
for k in characterSentences:
    print("----------------------------------------------")
    print(k)
    print("----------------------------------------------")
    print(type(characterSentences[k]))
    print("----------------------------------------------")
    for x in characterSentences[k]:
        print(x)
        print("----------------------------------------------")

----------------------------------------------
Privet Drive
----------------------------------------------
<class 'list'>
----------------------------------------------
/ 




THE BOY WHO LIVED 

Mr. and Mrs. Dursley, of number four, Privet Drive, 
were proud to say that they were perfectly normal, 
thank you very much.
----------------------------------------------
There was a tabby cat standing on the corner 
of Privet Drive, but there wasn’t a map in sight.
----------------------------------------------
It was staring down Privet Drive as though 
it were waiting for something.
----------------------------------------------
He clicked it once, and twelve balls of light 
sped back to their street lamps so that Privet Drive 
glowed suddenly orange and he could make out a 
tabby cat slinking around the corner at the other end 
of the street.
----------------------------------------------
A breeze ruffled the neat hedges of Privet Drive, which 
lay silent and tidy under the inky sky, the

Dudley.
----------------------------------------------
Ten years ago, there had been 
lots of pictures of what looked like a large pink beach 
ball wearing different-colored bonnets — but Dudley 
Dursley was no longer a baby, and now the 
photographs showed a large blond boy riding his first 
bicycle, on a carousel at the fair, playing a computer 
game with his father, being hugged and kissed by his 
mother.
----------------------------------------------
“Nothing, nothing ...” 

Dudley’s birthday — how could he have forgotten?
----------------------------------------------
The table was almost hidden beneath all 
Dudley’s birthday presents.
----------------------------------------------
It looked as though 
Dudley had gotten the new computer he wanted, not 
to mention the second television and the racing bike.
----------------------------------------------
Exactly why Dudley wanted a racing bike was a 
mystery to Harry, as Dudley was very fat and hated 
exercise — unless of course it i

squeeze him to death.
----------------------------------------------
A 
tiny man in a violet top hat had bowed to him once 
while out shopping with Aunt Petunia and Dudley.
----------------------------------------------
Everybody knew that 
Dudley’s gang hated that odd Harry Potter in his 
baggy old clothes and broken glasses, and nobody 
liked to disagree with Dudley’s gang.
----------------------------------------------
By the time he 
was allowed out of his cupboard again, the summer 
holidays had started and Dudley had already broken 
his new video camera, crashed his remote control 
airplane, and, first time out on his racing bike, 
knocked down old Mrs. Figg as she crossed Privet 
Drive on her crutches.
----------------------------------------------
Harry was glad school was over, but there was no 
escaping Dudley’s gang, who visited the house every 
single day.
----------------------------------------------
Piers, Dennis, Malcolm, and Gordon were 
all big and stupid, but as Dudl

----------------------------------------------
Uncle Vernon said sharply, “Don’t 
touch anything he gives you, Dudley.” 

The giant chuckled darkly.
----------------------------------------------
He’d spent his life being clouted by 
Dudley, and bullied by Aunt Petunia and Uncle 
Vernon; if he was really a wizard, why hadn’t they 
been turned into warty toads every time they’d tried 
to lock him in his cupboard?
----------------------------------------------
If he’d once defeated the 
greatest sorcerer in the world, how come Dudley had 
always been able to kick him around like a football?
----------------------------------------------
chased by 
Dudley’s gang, he had somehow found himself out of 
their reach .
----------------------------------------------
and the very last time Dudley had hit him, 



Page | 64 Harry Potter and the Philosophers Stone - J.K.
----------------------------------------------
Hagrid seized his 
umbrella and whirled it over his head, “NEVER — ” he 
thundered

and things that wouldn’t fit into his first bedroom.
----------------------------------------------
He mustn’t wake the Dursleys.
----------------------------------------------
The Dursleys 
ducked, but Harry leapt into the air trying to catch 
one — 

“Out!
----------------------------------------------
Of course, his birthdays were never exactly 
fun — last year, the Dursleys had given him a coat 
hanger and a pair of Uncle Vernon’s old socks.
----------------------------------------------
He lay and 
watched his birthday tick nearer, wondering if the 
Dursleys would remember at all, wondering where the 
letter writer was now.
----------------------------------------------
“Anyway — Harry,” said the giant, turning his back 
on the Dursleys, “a very happy birthday to yeh.
----------------------------------------------
“Sorry?” barked Hagrid, turning to stare at the 
Dursleys, who shrank back into the shadows.
----------------------------------------------
The Dursleys were cowering ag

...

door, away from Filch’s voice.
----------------------------------------------
Neville’s robes had 
barely whipped round the corner when they heard 
Filch enter the trophy room.
----------------------------------------------
They could hear Filch getting 
nearer.
----------------------------------------------
“RUN!” Harry yelled, and the four of them sprinted 
down the gallery, not looking back to see whether 
Filch was following — they swung around the 
doorpost and galloped down one corridor then 
another, Harry in the lead, without any idea where 
they were or where they were going — they ripped 
through a tapestry and found themselves in a hidden 
passageway, hurtled along it and came out near their 
Charms classroom, which they knew was miles from 
the trophy room.
----------------------------------------------
He was never going to meet 
you — Filch knew someone was going to be in the 
trophy room, Malfoy must have tipped him off.” 

Harry thought she was probably right, but he wa

a free shot at the goal posts for Gryffindor.
----------------------------------------------
----------------------------------------------
Fang
----------------------------------------------
<class 'list'>
----------------------------------------------
Then Hagrid ’s 
voice rang out, saying, “Back, Fang — back.” 

Hagrid ’s big, hairy face appeared in the crack as he 
pulled the door open.
----------------------------------------------
“Back, Fang.” 

He let them in, struggling to keep a hold on the collar 
of an enormous black boarhound.
----------------------------------------------
“Make yerselves at home,” said Hagrid, letting go of 
Fang, who bounded straight at Ron and started 
licking his ears.
----------------------------------------------
Like Hagrid, Fang was clearly not as 
fierce as he looked.
----------------------------------------------
Fang rested his head on 
Harry’s knee and drooled all over his robes.
----------------------------------------------
Harry and Ron were

dark, we might lose it.
----------------------------------------------
No sign of the Snitch 
yet, eh?” 

“Nope,” said Ron.
----------------------------------------------
Rowling 




Way up above them, Harry was gliding over the game, 
squinting about for some sign of the Snitch.
----------------------------------------------
“Keep out of the way until you catch sight of the 
Snitch,” Wood had said.
----------------------------------------------
Now he was 
back to staring around for the Snitch.
----------------------------------------------
“Slytherin in possession,” Lee Jordan was saying, 
“Chaser Pucey ducks two Bludgers, two Weasleys, 
and Chaser Bell, and speeds toward the — wait a 
moment — was that the Snitch?” 

A murmur ran through the crowd as Adrian Pucey 
dropped the Quaffle, too busy looking over his 
shoulder at the flash of gold that had passed his left 
ear.
----------------------------------------------
Neck and neck they 
hurtled toward the Snitch — all the Chasers s

and stepped underneath it themselves.
----------------------------------------------
“Well, well, well,” he whispered, “we are in trouble.” 

They’d left the Invisibility Cloak on top of the tower.
----------------------------------------------
Add 
Norbert and the Invisibility Cloak, and they might as 
well be packing their bags already.
----------------------------------------------
When Harry pulled back his sheets, he found his 
Invisibility Cloak folded neatly underneath them.
----------------------------------------------
“I’ll use the Invisibility Cloak,” said Harry.
----------------------------------------------
“You’ll understand later, Neville,” said Ron as they 
stepped over him and pulled on the Invisibility Cloak.
----------------------------------------------
When he had found his voice again, 
Harry said, “And the Invisibility Cloak — do you know 
who sent it to me?” 



Page | 336 Harry Potter and the Philosophers Stone - J.K.
-------------------------------------------

the trees blocked their view.
----------------------------------------------
A centaur was 
standing over him, not Ronan or Bane; this one 
looked younger; he had white-blond hair and a 
palomino body.
----------------------------------------------
Ronan and Bane came 
bursting through the trees, their flanks heaving and 
sweaty.
----------------------------------------------
“Firenze!” Bane thundered.
----------------------------------------------
The quicker he leaves this forest, the 
better.” 

“What have you been telling him?” growled Bane.
----------------------------------------------
Bane kicked his back legs in anger.
----------------------------------------------
“Do you not see that unicorn?” Firenze bellowed at 
Bane.
----------------------------------------------
I set 
myself against what is lurking in this forest, Bane, 
yes, with humans alongside me if I must.” 



Page | 289 Harry Potter and the Philosophers Stone - J.K.
----------------------------------------------
R

In [191]:
def compareLists1(sentenceList1, majorCharacters1):
    """
    Compares the list of sentences with the character names and returns
    sentences that include names.
    """
    characterSentence1 = set()
    for sentence in sentenceList1:
        for name in majorCharacters1:
            if re.search(name,sentence,re.IGNORECASE):
                characterSentence1.add(sentence)
    return characterSentence1

In [196]:
for k in characterSentences:
    characterDescription = defaultdict(set)
    sen=compareLists1(characterSentences[k],lines_list)
    print(sen)
    characterDescription[k]=sen

{'Potter, The Smallest \nBedroom, 4 Privet Drive — ’ ” \n\nWith a strangled cry, Uncle Vernon leapt from his \nseat and ran down the hall, Harry right behind him.', 'He clicked it once, and twelve balls of light \nsped back to their street lamps so that Privet Drive \nglowed suddenly orange and he could make out a \ntabby cat slinking around the corner at the other end \nof the street.', 'It was staring down Privet Drive as though \nit were waiting for something.', 'Potter \n\nThe Cupboard under the Stairs \n4 Privet Drive \nLittle Whinging \nSurrey \n\nThe envelope was thick and heavy, made of yellowish \nparchment, and the address was written in emerald- \ngreen ink.', 'Rowling \n\n\n\n\n\n\nTHE VANASHIG GLASS \n\nNearly ten years had passed since the Dursleys had \nwoken up to find their nephew on the front step, but \nPrivet Drive had hardly changed at all.', 'A breeze ruffled the neat hedges of Privet Drive, which \nlay silent and tidy under the inky sky, the very last \n\nPage | 

{'Dudley had been \naccepted at Uncle Vernon’s old private school, \nSmeltings.', 'Instead he said, as casually as he could, “Their son — \nhe’d be about Dudley’s age now, wouldn’t he?” \n\n“I suppose so,” said Mrs. Dursley stiffly.', '“Dudley — go — just go.” \n\nHarry walked round and round his new room.', 'I’m \ngoing to have a lot of fun with Dudley this summer...” \n\n\n\nPage | 348 Harry Potter and the Philosophers Stone - J.K.', 'It was even \nworth being with Dudley and Piers to be spending the \nday somewhere that wasn’t school, his cupboard, or \nMrs. Figg’s cabbage-smelling living room.', 'Dudley thought this was very \nfunny.', 'By the time he \nwas allowed out of his cupboard again, the summer \nholidays had started and Dudley had already broken \nhis new video camera, crashed his remote control \nairplane, and, first time out on his racing bike, \nknocked down old Mrs. Figg as she crossed Privet \nDrive on her crutches.', 'Dudley’s favorite punching bag was Harry, \nbut h

{'The Dursleys had \nreceived a very angry letter from Harry’s headmistress \ntelling them Harry had been climbing school \nbuildings.', 'When he had been younger, Harry had dreamed and \ndreamed of some unknown relation coming to take \nhim away, but it had never happened; the Dursleys \nwere his only family.', 'If Harry hadn’t \nknown that the Dursleys had no sense of humor, he \nmight have thought so; yet somehow, even though \neverything Hagrid had told him so far was \nunbelievable, Harry couldn’t help trusting him.', 'He checked his Hogwarts list yet \nagain to make sure he had everything he needed, saw \nthat Hedwig was shut safely in her cage, and then \npaced the room, waiting for the Dursleys to get up.', 'These \nfantastic party favors were nothing like the feeble \nMuggle ones the Dursleys usually bought, with their \nlittle plastic toys and their flimsy paper hats inside.', 'If I get caught before I can get to the Stone, well, \nI’ll have to go back to the Dursleys and wai

{'The \nhut was full of sunlight, the storm was over, Hagrid \nhimself was asleep on the collapsed sofa, and there \n\n\n\nPage | 68 Harry Potter and the Philosophers Stone - J.K.', 'Page | 204 Harry Potter and the Philosophers Stone - J.K.', '“Everywhere else is full.” \n\nPage | 109 Harry Potter and the Philosophers Stone - J.K.', '“Knuts?” \n\n“The little bronze ones.” \n\nHarry counted out five little bronze coins, and the owl \nheld out his leg so Harry could put the money into a \n\n\n\nPage | 69 Harry Potter and the Philosophers Stone - J.K.', 'Harry, still cheering, nudged Ron in the ribs \nand pointed at Malfoy, who couldn’t have looked more \nPage | 344 Harry Potter and the Philosophers Stone - J.K.', 'Mr. Dursley blinked and stared at \nPage | 3 Harry Potter and the Philosophers Stone - J.K.', 'He was tall, thin, and very old, judging by the \nsilver of his hair and beard, which were both long \n\nPage | 9 Harry Potter and the Philosophers Stone - J.K.', 'But all he’d tried 

{'The \nhut was full of sunlight, the storm was over, Hagrid \nhimself was asleep on the collapsed sofa, and there \n\n\n\nPage | 68 Harry Potter and the Philosophers Stone - J.K.', 'Page | 204 Harry Potter and the Philosophers Stone - J.K.', '“Everywhere else is full.” \n\nPage | 109 Harry Potter and the Philosophers Stone - J.K.', '“Knuts?” \n\n“The little bronze ones.” \n\nHarry counted out five little bronze coins, and the owl \nheld out his leg so Harry could put the money into a \n\n\n\nPage | 69 Harry Potter and the Philosophers Stone - J.K.', 'Harry, still cheering, nudged Ron in the ribs \nand pointed at Malfoy, who couldn’t have looked more \nPage | 344 Harry Potter and the Philosophers Stone - J.K.', 'Mr. Dursley blinked and stared at \nPage | 3 Harry Potter and the Philosophers Stone - J.K.', 'He was tall, thin, and very old, judging by the \nsilver of his hair and beard, which were both long \n\nPage | 9 Harry Potter and the Philosophers Stone - J.K.', 'But all he’d tried 

...

{'While the \nGryffindor common room and the Great Hall had \nroaring fires, the drafty corridors had become icy and \na bitter wind rattled the windows in the classrooms. \nWorst of all were Professor Snape’s classes down in \nthe dungeons, where their breath rose in a mist \nbefore them and they kept as close as possible to \ntheir hot cauldrons.', 'As the owls flooded into the Great Hall as usual, \neveryone’s attention was caught at once by a long, \nthin package carried by six large screech owls.', '“Now, form a line,” Professor McGonagall told the first \nyears, “and follow me.” \n\nFeeling oddly as though his legs had turned to lead, \nHarry got into line behind a boy with sandy hair, with \nRon behind him, and they walked out of the chamber, \nback across the hall, and through a pair of double \ndoors into the Great Hall.', 'I therefore award ten points to Mr. Neville \nLongbottom.” \n\nSomeone standing outside the Great Hall might well \nhave thought some sort of explosion had

{'your \nfather used it mainly for sneaking off to the kitchens \nto steal food when he was here.” \n\n“And there’s something else ...” \n\n“Fire away.” \n\n“Quirrell said Snape — ” \n\n“Professor Snape, Harry.” \n\n“Yes, him — Quirrell said he hates me because he \nhated my father.', 'At times, he even wondered whether Snape \nwas following him, trying to catch him on his own.', '“Hagrid \ntold that stranger how to get past Fluffy, and it was \neither Snape or Voldemort under that cloak — it \nmust’ve been easy, once he’d got Hagrid drunk.', '“One of us has got to keep an eye on Snape \n— wait outside the staffroom and follow him if he \nleaves it.', '“Snape’s Head of Slytherin House.', '“I wondered whether I’d be \nmeeting you here, Potter.” \n\n“But I thought — Snape — ” \n\n“Severus?” Quirrell laughed, and it wasn’t his usual \nquivering treble, either, but cold and sharp.', 'Come on — I shall see Professor Snape about you, \nMalfoy!” \n\nThe steep spiral staircase up to the top of

{'“Follow me,” said Filch, lighting a lamp and leading \nthem outside.', 'Filch wouldn’t help us if his life \ndepended on it, he’s too friendly with Snape, and the \nmore students get thrown out, the better, he’ll think.', 'He passed Filch in the doorway; Filch’s pale, \nwild eyes looked straight through him, and Harry \nslipped under Filch’s outstretched arm and streaked \noff up the corridor, the book’s shrieks still ringing in \nhis ears.', 'Neville’s robes had \nbarely whipped round the corner when they heard \nFilch enter the trophy room.', 'Naughty, naughty, you’ll get caughty.” \n\n“Not if you don’t give us away, Peeves, please.” \n\n“Should tell Filch, I should,” said Peeves in a sanity \nvoice, but his eyes glittered wickedly.', 'Filch must have hurried off to look for them \nsomewhere else, because they didn’t see him \nanywhere, but they hardly cared — all they wanted to \ndo was put as much space as possible between them \nand that monster.', 'Harry and Ron were delighted 

{'“Slytherin in possession — Flint with the Quaffle — \npasses Spinnet — passes Bell — hit hard in the face \nby a Bludger, hope it broke his nose — only joking, \nProfessor — Slytherins score — oh no ...” \n\nThe Slytherins were cheering.', 'You’ve got to weave in and out \nof the Chasers, Beaters, Bludgers, and Quaffle to get \nit before the other team’s Seeker, because whichever \nSeeker catches the Snitch wins his team an extra \nhundred and fifty points, so they nearly always win.', 'And \nyou don’t have to worry about the Quaffle or the \nBludgers — ” \n\n“ — unless they crack my head open.” \n\n\n\nPage | 189 Harry Potter and the Philosophers Stone - J.K.', 'So — think you’ve \ngot all that?” \n\n“Three Chasers try and score with the Quaffle; the \nKeeper guards the goal posts; the Beaters keep the \nBludgers away from their team,” Harry reeled off.', '“These two are the Bludgers.” \n\nHe showed Harry two identical balls, jet black and \nslightly smaller than the red Quaffle.', 

{'Ron appeared out \nof nowhere as he pulled off Harry’s Invisibility Cloak.', '“Well, well, well,” he whispered, “we are in trouble.” \n\nThey’d left the Invisibility Cloak on top of the tower.', 'Rowling \n\n\n\n\n\n\nNICHOLAS FLAMBL \n\nDumbledore had convinced Harry not to go looking \nfor the Mirror of Erised again, and for the rest of the \nChristmas holidays the Invisibility Cloak stayed \nfolded at the bottom of his trunk.', '“It’s an Invisibility Cloak,” said Ron, a look of awe on \nhis face.', '“Bye-bye, Norbert!” Hagrid sobbed, as Harry and \nHermione covered the crate with the Invisibility Cloak \nand stepped underneath it themselves.', '“You’ll understand later, Neville,” said Ron as they \nstepped over him and pulled on the Invisibility Cloak.', 'Not until he climbed into bed was he free to \nthink about it: the Invisibility Cloak and whoever had \nsent it.', 'And we have got \nthe Invisibility Cloak, Malfoy doesn’t know about \nthat.” \n\nThey found Fang the boarhound si

In [193]:
for k in characterSentences:
    print(k)
    print(characterDescription[k])

Privet Drive
set()
Mr. Dursley
set()
Dudley
set()
Dursleys
set()
Harry Potter
set()
Potter
set()
Philosophers Stone
set()
Harry
set()
Page
set()
House
set()
Muggles
set()
Muggle
set()
Dumbledore
set()
McGonagall
set()
Professor
set()
Professor McGonagall
set()
Everyone
set()
Voldemort
set()
Madam Pomfrey
set()
Hagrid
set()
London
set()
Aunt Petunia
set()
Uncle Vernon
set()
Wood
set()
Hogwarts
set()
Mr. Potter
set()
Sorcerer
set()
Magic
set()
Gringotts
set()
Quirrell
set()
Griphook
set()
Quidditch
set()
Slytherin
set()
Hufflepuff
set()
Mr. Ollivander
set()
Hedwig
set()
Percy
set()
Fred
set()
George
set()
Ron
set()
Neville
set()
Weasley
set()
Charlie
set()
Flamel
set()
Hermione Granger
set()
Hermione
set()
Gryffindor
set()
Crabbe
set()
Goyle
set()
Malfoy
set()
Great Hall
set()
Peeves
set()
Seamus
set()
Snape
set()
Filch
set()
Madam Hooch
set()
Fang
set()
Quaffle
set()
Snitch
set()
Fluffy
set()
Invisibility Cloak
set()
Norbert
set()
Ronan
set()
Bane
set()
Firenze
{'Rowling \n\n\n\n\nAnd F

In [195]:
for k in characterSentences:
    print(k)
    print("---------------------------------------------------")
    for sentence in characterSentences[k]:
        for name in lines_list:
            if re.search(r"\b(?=\w)%s\b(?!\w)" % re.escape(name),
                         sentence,
                         re.IGNORECASE):
                print(sentence)
                print(name)

Privet Drive
---------------------------------------------------
He clicked it once, and twelve balls of light 
sped back to their street lamps so that Privet Drive 
glowed suddenly orange and he could make out a 
tabby cat slinking around the corner at the other end 
of the street.
Light
Potter 

The Cupboard under the Stairs 
4 Privet Drive 
Little Whinging 
Surrey 

The envelope was thick and heavy, made of yellowish 
parchment, and the address was written in emerald- 
green ink.
thick
Mr. Dursley
---------------------------------------------------
At half past eight, Mr. Dursley picked up his 
briefcase, pecked Mrs. Dursley on the cheek, and 
tried to kiss Dudley good-bye but missed, because 
Dudley was now having a tantrum and throwing his 
cereal at the walls.
cheek
At half past eight, Mr. Dursley picked up his 
briefcase, pecked Mrs. Dursley on the cheek, and 
tried to kiss Dudley good-bye but missed, because 
Dudley was now having a tantrum and throwing his 
cereal at the walls

in the face.
Red
“That’s two less than last year.” 

“Darling, you haven’t counted Auntie Marge’s present, 
see, it’s here under this big one from Mommy and 
Daddy.” 

“All right, thirty-seven then,” said Dudley, going red 
in the face.
face
“That’s two less than last year.” 

“Darling, you haven’t counted Auntie Marge’s present, 
see, it’s here under this big one from Mommy and 
Daddy.” 

“All right, thirty-seven then,” said Dudley, going red 
in the face.
big
Harry, who could see a huge Dudley 
tantrum coming on, began wolfing down his bacon as 
fast as possible in case Dudley turned the table over.
huge
’Atta boy, Dudley!” He ruffled Dudley’s hair.
hair
’Atta boy, Dudley!” He ruffled Dudley’s hair.
hair
Dudley’s mouth fell open in horror, but Harry’s heart 
gave a leap.
mouth
t-t-to come!” Dudley 
yelled between huge, pretend sobs.
huge
“He always sp- 
spoils everything!” He shot Harry a nasty grin through 
the gap in his mothers arms. 

Just then, the doorbell rang — “Oh, good Lord

with orange puff balls).
Old
Another time, Aunt Petunia had been trying to force 
him into a revolting old sweater of Dudley’s (brown 
with orange puff balls).
Brown
He 
turned right around in his seat and yelled at Harry, 
his face like a gigantic beet with a mustache: 
“MOTORCYCLES DONT FLY!” 

Dudley and Piers sniggered.
Gigantic
He 
turned right around in his seat and yelled at Harry, 
his face like a gigantic beet with a mustache: 
“MOTORCYCLES DONT FLY!” 

Dudley and Piers sniggered.
face
It 
wasn’t bad, either, Harry thought, licking it as they 
watched a gorilla scratching its head who looked 
remarkably like Dudley, except that it wasn’t blond.
Blond
It 
wasn’t bad, either, Harry thought, licking it as they 
watched a gorilla scratching its head who looked 
remarkably like Dudley, except that it wasn’t blond.
head
Dudley and Piers wanted to see huge, 
poisonous cobras and thick, man-crushing pythons.
thick
Dudley and Piers wanted to see huge, 
poisonous cobras and thick, man-c

had sat on it.
Small
The month-old video 
camera was lying on top of a small, working tank 
Dudley had once driven over the next door neighbor’s 
dog; in the corner was Dudley’s first-ever television 
set, which he’d put his foot through when his favorite 
program had been canceled; there was a large 
birdcage, which had once held a parrot that Dudley 
had swapped at school for a real air rifle, which was 
up on a shelf with the end all bent because Dudley 
had sat on it.
large
The month-old video 
camera was lying on top of a small, working tank 
Dudley had once driven over the next door neighbor’s 
dog; in the corner was Dudley’s first-ever television 
set, which he’d put his foot through when his favorite 
program had been canceled; there was a large 
birdcage, which had once held a parrot that Dudley 
had swapped at school for a real air rifle, which was 
up on a shelf with the end all bent because Dudley 
had sat on it.
foot
Uncle Vernon had to wrestle Dudley to the ground to 
get

teachers wig blue.
Blue
If I get caught before I can get to the Stone, well, 
I’ll have to go back to the Dursleys and wait for 
Voldemort to find me there, it’s only dying a bit later 
than I would have, because I’m never going over to 
the Dark Side!
Dark
Harry Potter
---------------------------------------------------
He was tall, thin, and very old, judging by the 
silver of his hair and beard, which were both long 

Page | 9 Harry Potter and the Philosophers Stone - J.K.
thin
He was tall, thin, and very old, judging by the 
silver of his hair and beard, which were both long 

Page | 9 Harry Potter and the Philosophers Stone - J.K.
thin
He was tall, thin, and very old, judging by the 
silver of his hair and beard, which were both long 

Page | 9 Harry Potter and the Philosophers Stone - J.K.
hair
He was tall, thin, and very old, judging by the 
silver of his hair and beard, which were both long 

Page | 9 Harry Potter and the Philosophers Stone - J.K.
long
He was tall, thin, and ve

...

net, had Malfoy by the ear.
hair
Rowling 




Professor McGonagall, in a tartan bathrobe and a hair 
net, had Malfoy by the ear.
hair
“I was trying to find you to warn you, I 
heard Malfoy saying he was going to catch you, he 
said you had a drag — ” 

Harry shook his head violently to shut Neville up, but 
Professor McGonagall had seen.
head
There’s blood 
all over the place, it must’ve bin staggerin’ around 
since last night at least.” 

“I want Fang,” said Malfoy quickly, looking at Fang’s 
long teeth.
long
There’s blood 
all over the place, it must’ve bin staggerin’ around 
since last night at least.” 

“I want Fang,” said Malfoy quickly, looking at Fang’s 
long teeth.
Long
There’s blood 
all over the place, it must’ve bin staggerin’ around 
since last night at least.” 

“I want Fang,” said Malfoy quickly, looking at Fang’s 
long teeth.
teeth
There’s blood 
all over the place, it must’ve bin staggerin’ around 
since last night at least.” 

“I want Fang,” said Malfoy quickly, lookin

and Seamus winked.
eye
Harry’s partner was 
Seamus Finnigan (which was a relief, because Neville 
had been trying to catch his eye).
eye
Harry’s partner was 
Seamus Finnigan (which was a relief, because Neville 
had been trying to catch his eye).
eye
Snape
---------------------------------------------------
Knows an awful lot 
about the Dark Arts, Snape.” 

Harry watched Snape for a while, but Snape didn’t 
look at him again.
Dark
Harry told the turban he didn’t want 
to be in Slytherin; it got heavier and heavier; he tried 
to pull it off but it tightened painfully — and there 
was Malfoy, laughing at him as he struggled with it — 
then Malfoy turned into the hook-nosed teacher, 
Snape, whose laugh became high and cold — there 
was a burst of green light and Harry woke, sweating 
and shaking.
Light
“Snape’s Head of Slytherin House.
head
Snape ’s lips curled into a sneer.
lips
Over the noise, Snape said, “And a point 
will be taken from Gryffindor House for your cheek, 
Potter.” 

Thin

bite Snape ’s leg off properly.
Straight
Unfortunately, while everyone else was 
running around looking for it, Snape, who already 
suspected me, went straight to the third floor to head 
me off — and not only did my troll fail to beat you to 
death, that three-headed dog didn’t even manage to 
bite Snape ’s leg off properly.
head
He caught Harry’s eye and Harry knew at once 
that Snape ’s feelings toward him hadn’t changed one 
jot.
eye
He caught Harry’s eye and Harry knew at once 
that Snape ’s feelings toward him hadn’t changed one 
jot.
eye
Filch
---------------------------------------------------
He would drop 
wastepaper baskets on your head, pull rugs from 
under your feet, pelt you with bits of chalk, or sneak 
up behind you, invisible, grab your nose, and screech, 
“GOT YOUR CONK!” 

Even worse than Peeves, if that was possible, was the 
caretaker, Argus Filch.
nose
He would drop 
wastepaper baskets on your head, pull rugs from 
under your feet, pelt you with bits of chalk, or

licking his ears.
Straight
“Make yerselves at home,” said Hagrid, letting go of 
Fang, who bounded straight at Ron and started 
licking his ears.
ears
Fang rested his head on 
Harry’s knee and drooled all over his robes.
head
Harry and Ron were delighted to hear Hagrid call 
Filch “that old git.” 

“An’ as fer that cat, Mrs. Norris, I’d like ter introduce 
her to Fang sometime.
old
Harry and Ron were delighted to hear Hagrid call 
Filch “that old git.” 

“An’ as fer that cat, Mrs. Norris, I’d like ter introduce 
her to Fang sometime.
Old
Rowling 




Hagrid came striding toward them out of the dark, 
Fang at his heel.
Dark
There’s blood 
all over the place, it must’ve bin staggerin’ around 
since last night at least.” 

“I want Fang,” said Malfoy quickly, looking at Fang’s 
long teeth.
long
There’s blood 
all over the place, it must’ve bin staggerin’ around 
since last night at least.” 

“I want Fang,” said Malfoy quickly, looking at Fang’s 
long teeth.
Long
There’s blood 
all over the

leg
Hagrid had Norbert packed and ready in a large crate.
large
Hagrid had Norbert packed and ready in a large crate.
large
Hagrid had Norbert packed and ready in a large crate.
large
Midnight ticked nearer as 
they heaved Norbert up the marble staircase in the 
entrance hall and along the dark corridors.
Dark
Rowling 




They slipped back down the spiral staircase, their 
hearts as light as their hands, now that Norbert was 
off them.
Light
Ronan
---------------------------------------------------
Rowling 




black-haired and -bodied and wilder-looking than 
Ronan.
Black
There’s a unicorn 
bin injured — would yeh know anythin’ about it?” 

Bane walked over to stand next to Ronan.
stand
A centaur was 
standing over him, not Ronan or Bane; this one 
looked younger; he had white-blond hair and a 
palomino body.
hair
A centaur was 
standing over him, not Ronan or Bane; this one 
looked younger; he had white-blond hair and a 
palomino body.
body
A centaur was 
standing over him, not Rona

In [201]:
from pycorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP()


def resolve(corenlp_output):
    """ Transfer the word form of the antecedent to its associated pronominal anaphor(s) """
    for coref in corenlp_output['corefs']:
        mentions = corenlp_output['corefs'][coref]
        antecedent = mentions[0]  # the antecedent is the first mention in the coreference chain
        for j in range(1, len(mentions)):
            mention = mentions[j]
            if mention['type'] == 'PRONOMINAL':
                # get the attributes of the target mention in the corresponding sentence
                target_sentence = mention['sentNum']
                target_token = mention['startIndex'] - 1
                # transfer the antecedent's word form to the appropriate token in the sentence
                corenlp_output['sentences'][target_sentence - 1]['tokens'][target_token]['word'] = antecedent['text']


def print_resolved(corenlp_output):
    """ Print the "resolved" output """
    possessives = ['hers', 'his', 'their', 'theirs']
    for sentence in corenlp_output['sentences']:
        for token in sentence['tokens']:
            output_word = token['word']
            # check lemmas as well as tags for possessive pronouns in case of tagging errors
            if token['lemma'] in possessives or token['pos'] == 'PRP$':
                output_word += "'s"  # add the possessive morpheme
            output_word += token['after']
            print(output_word, end='')


text = "Tom and Jane are good friends. They are cool. He knows a lot of things and so does she. His car is red, but " \
       "hers is blue. It is older than hers. The big cat ate its dinner."

output = nlp.annotate(text, properties= {'annotators':'dcoref','outputFormat':'json','ner.useSUTime':'false'})

resolve(output)

print('Original:', text)
print('Resolved: ', end='')
print_resolved(output)

TypeError: __init__() missing 1 required positional argument: 'server_url'

In [202]:
!pip install neuralcoref
#!pip install pycorenlp

  Using cached https://files.pythonhosted.org/packages/4a/1c/6359be64e8301b84160f6f6f7936bbfaaa5e9a4eab6cbc681db07600b949/tqdm-4.45.0-py2.py3-none-any.whl
  Found existing installation: tqdm 4.31.1
    Uninstalling tqdm-4.31.1:
      Successfully uninstalled tqdm-4.31.1


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
#!python -m spacy download en
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [5]:
import neuralcoref
neuralcoref.add_to_pipe(nlp)

C:\Users\Dell\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
C:\Users\Dell\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
C:\Users\Dell\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [ ]:
doc = nlp(u'My sister has a dog. She loves him.')

In [6]:
nlp.remove_pipe("neuralcoref")

('neuralcoref', <neuralcoref.neuralcoref.NeuralCoref at 0x22b7cdbdb08>)

In [7]:
coref = neuralcoref.NeuralCoref(nlp.vocab, greedyness=0.75)

In [8]:
nlp.add_pipe(coref, name='neuralcoref')

In [ ]:
doc = nlp(u'Deepika has a dog. She loves him.')